In [4]:
class meta:
    def __init__(self, data):
        # Check if data is a dictionary
        if isinstance(data, dict):
            for key, value in data.items():
                setattr(self, key, value)
        else:
            raise ValueError("Expected a dictionary")

In [5]:
meta = meta({
    "source":[
        "bronze.dim1",
        "bronze.dim2"
    ],
    "target":[
        "silver.dim2",
        "silver.dim3",
        "silver.dim4"
    ]
})

In [23]:
pip install pandas

     ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.6 MB 1.3 MB/s eta 0:00:10
     ---------------------------------------- 0.1/11.6 MB 1.4 MB/s eta 0:00:08
      --------------------------------------- 0.2/11.6 MB 2.1 MB/s eta 0:00:06
     - -------------------------------------- 0.5/11.6 MB 2.9 MB/s eta 0:00:04
     -- ------------------------------------- 0.8/11.6 MB 3.4 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/11.6 MB 4.6 MB/s eta 0:00:03
     ------- -------------------------------- 2.2/11.6 MB 7.0 MB/s eta 0:00:02
     --------- ------------------------------ 2.7/11.6 MB 7.5 MB/s eta 0:00:02
     ------------ --------------------------- 3.6/11.6 MB 8.8 MB/s eta 0:00:01
     -------------- ------------------------- 4.1/11.6 MB 8.7 MB/s eta 0:00:01
     ------------------- -------------------- 5.7/11.6 MB 11.1 MB/s eta 0:00:01
     ---------------------- ----------------- 6.5/11.6 MB 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: c:\Users\vanty\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [ ]:
import re
import pandas as pd
import boto3
from io import StringIO

def read_script_from_s3(bucket_name, object_key):
    s3_client = boto3.client('s3')
    response = s3_client.get_object(Bucket=bucket_name, Key=object_key)
    return response['Body'].read().decode('utf-8')

# Specify your S3 bucket and object key (file path)
bucket_name = 'your-bucket-name'
object_key = 'path/to/your/delta.gold.testing.py'

# Read the PySpark script from S3
pyspark_script = read_script_from_s3(bucket_name, object_key)

# Extract SQL queries
sql_query_pattern = r"(?<=spark.sql\(f\"\"\"\n)(.*?)(?=\"\"\"\))"
sql_queries_matches = re.findall(sql_query_pattern, pyspark_script, re.DOTALL)

all_source_tables = set()

# Process each SQL query
for sql_query in sql_queries_matches:
    # Regex pattern to find tables in FROM and JOIN clauses
    combined_pattern = r"from\s+([a-zA-Z0-9_.]+)|(?:left join|right join|inner join|full outer join|cross join|left anti join|right anti join)\s+([a-zA-Z0-9_.]+)"
    combined_matches = re.findall(combined_pattern, sql_query, re.IGNORECASE)

    # Add the found tables to the set for uniqueness
    for match in combined_matches:
        if match[0]:  # Main source table
            all_source_tables.add(match[0])
        if match[1]:  # Join tables
            all_source_tables.add(match[1])

# Get the target table name from the object key (file name)
target_table_name = object_key.split('/')[-1].split('.')[0]  # Remove file extension

# Prepare DataFrame
source_table_list = list(all_source_tables)
df = pd.DataFrame({'source_table': source_table_list, 'target_table': target_table_name})

# Display the DataFrame
print(df)


In [28]:
import re
import pandas as pd
import os

def read_script_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Path to your PySpark script file
script_file_path = './delta.gold.testing.py'

# Read the PySpark script
pyspark_script = read_script_from_file(script_file_path)



sql_query_pattern = r"(?<=spark.sql\(f\"\"\"\n)(.*?)(?=\"\"\"\))"
sql_queries_matches = re.findall(sql_query_pattern, pyspark_script, re.DOTALL)

all_source_tables = set()

# Process each SQL query
for idx, sql_query in enumerate(sql_queries_matches):


    # Regex pattern to find tables in FROM and JOIN clauses
    combined_pattern = r"from\s+([a-zA-Z0-9_.]+)|(?:left join|right join|inner join|full outer join|cross join|left anti join|right anti join)\s+([a-zA-Z0-9_.]+)"
    combined_matches = re.findall(combined_pattern, sql_query, re.IGNORECASE)

    # Add the found tables to the set for uniqueness
    for match in combined_matches:
        if match[0]:  # Main source table
            all_source_tables.add(match[0])
        if match[1]:  # Join tables
            all_source_tables.add(match[1])

target_table_name = script_file_path.split('/')[-1].replace('.py','')  # Remove file extension

# Prepare DataFrame
source_table_list = list(all_source_tables)
df = pd.DataFrame({'source_table': source_table_list, 'target_table': target_table_name})


In [29]:
df

,source_table,target_table
0,delta.GOLD.D_TRANSACTION_TYPE,delta.gold.testing
1,delta.GOLD.D_CUSTOMER,delta.gold.testing
2,delta.SILVER.FHO1_FUNDS_TRANSFER000,delta.gold.testing
3,delta.GOLD.D_PMH_TRANSACTION_TYPE,delta.gold.testing
4,delta.bronze.FBNK_TRANSACTION,delta.gold.testing
5,delta.GOLD.D_ACCOUNT,delta.gold.testing
6,delta.SILVER.PMH_TRANSACTION_TYPE,delta.gold.testing
7,delta.SILVER.FHO1_STMT_ENTRY,delta.gold.testing
8,delta.GOLD.D_COMPANY,delta.gold.testing


In [21]:
all_source_tables

{'delta.GOLD.D_ACCOUNT',
 'delta.GOLD.D_COMPANY',
 'delta.GOLD.D_CUSTOMER',
 'delta.GOLD.D_PMH_TRANSACTION_TYPE',
 'delta.GOLD.D_TRANSACTION_TYPE',
 'delta.SILVER.FHO1_FUNDS_TRANSFER000',
 'delta.SILVER.FHO1_STMT_ENTRY',
 'delta.SILVER.PMH_TRANSACTION_TYPE',
 'delta.bronze.FBNK_TRANSACTION'}

In [ ]:
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from datetime import datetime

# Define the dependency string
dependency_string = "a,b|b,c|a,d"

# Step 1: Parse the dependency string
dependencies = dependency_string.split('|')
task_dependencies = [dep.split(',') for dep in dependencies]

# Step 2: Create the DAG
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 10, 1),
}

with DAG('dynamic_dependency_dag', default_args=default_args, schedule_interval='@daily') as dag:
    # Create a dictionary to hold task references
    tasks = {}

    # Create tasks based on unique identifiers
    for task_pair in task_dependencies:
        for task_id in task_pair:
            if task_id not in tasks:
                tasks[task_id] = DummyOperator(task_id=task_id)

    # Set task dependencies
    for task_pair in task_dependencies:
        if len(task_pair) == 2:
            tasks[task_pair[0]] >> tasks[task_pair[1]]

# This DAG will now reflect the dependencies:
# a >> b
# b >> c
# a >> d

In [ ]:
import pandas as pd
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from datetime import datetime

# Step 1: Create a DataFrame from the dependency string
dependency_string = "a,b|b,c|a,d"
dependencies = [dep.split(',') for dep in dependency_string.split('|')]

# Create a DataFrame
df_dependencies = pd.DataFrame(dependencies, columns=['task_from', 'task_to'])

# Step 2: Generate the Airflow DAG
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 10, 1),
}

with DAG('dynamic_dependency_dag', default_args=default_args, schedule_interval='@daily') as dag:
    tasks = {}

    # Create tasks based on unique identifiers
    for _, row in df_dependencies.iterrows():
        from_task = row['task_from']
        to_task = row['task_to']

        if from_task not in tasks:
            tasks[from_task] = DummyOperator(task_id=from_task)

        if to_task not in tasks:
            tasks[to_task] = DummyOperator(task_id=to_task)

        # Set the dependency
        tasks[from_task] >> tasks[to_task]

# The DAG will have the following dependencies:
# a >> b
# b >> c
# a >> d

In [10]:
meta.target

['silver.dim2', 'silver.dim3', 'silver.dim4']

In [7]:
meta.source[0]

'bronze.dim1'

In [11]:
result = []
for src in meta.source:
    for tgt in meta.target:
        result.append([src, tgt])

In [12]:
result

[['bronze.dim1', 'silver.dim2'],
 ['bronze.dim1', 'silver.dim3'],
 ['bronze.dim1', 'silver.dim4'],
 ['bronze.dim2', 'silver.dim2'],
 ['bronze.dim2', 'silver.dim3'],
 ['bronze.dim2', 'silver.dim4']]